# Сравнение различных библиотек для парсинга на Python

Цель исследования понять особенности работы двух библиотек для веб-парсинга: `BeautifulSoup` и `Scrapy`. Также будут проведены замеры работы парсеров (время и память), написанных с помощью данных библиотек, на двух разных сайтах: Хабр и Циан, чтобы выяснить, какая библиотека является более мощной.

## 1. Какие особенности сложны с точки зрения парсинга 


Первые сложности возникают при получении самой html-страницы.
1. На многих сайтах включена защита от ботов, поэтому надо обходить ограничение на количество запросов, а также проверка на то, кто подключается, поэтому следует прописывать UserAgent.
2. Если контент веб-страницы формируется методами JavaScript, то данные могут генерироваться автоматически при каждом запросе, соответственно, прописать верные атрибуты для поиска будет не так просто.
3. Веб-страница может передаваться с использованием компрессии (сжатия), в результате чего мы получим непонятные бинарные данные вместо нужной странички.
4. Сайты могут использовать необычную кодировку, тогда при скачивании страницы мы получим непонятные символы, вместо html-кода
5. На некоторых сайтах, чтобы получить информацию, надо предварительно пройти аутентифакцию, что тоже добавляет сложности при парсинге.
6. Если у сайта слишком низкая скорость зашгрузки, то запросы могут приходить быстрее, чем они могут обрабатываться, в следствие чего будут возникать ошибки.

Далее при непосредственном парсинге и построении синтаксического дерева разбора могут возникнуть следующие проблемы:
1. Не все парсеры умеют работать с испорченными html-файлами. Например, lxml может распарсить только высококачественный файл. Соответственно, если у нас в файле есть ошибка, то будут проблемы.
2. Не все парсеры знают о наличии самозакрывающихся тегов (Например, bs4).
3. У всех сайтов очень разная структура html-документа, в следствие чего, когда нужно спарсить несколько веб-сайтов, для каждого из них придется создать свой парсер.
4. Веб-страницы часто меняют дизайн, поэтому парсер придется часто обновлять и поддерживать актуальным



## 2. Ключевые особенности каждой из библиотек (BeautifulSoup, Scrapy)

### Scrapy
1. Работает на разных платформах (Linux, Windows, Mac OS)
2. Имеет встроенные селекторы `XPath` и `CSS`
3. Одна из самых быстрых библиотек
4. Потребляет меньше памяти
5. Построен на основе Twisted (асинхронной сетевой структуры)
6. Извлекает данные не только с одной страницы, а может "ползти" по другим страницам от корневого URL 
7. Можно определить правила для обхода URL

### BeautifulSoup 
1. Одна из самых распространенных библиотек, в следствие чего имеет хорошую документацию
2. Хорошо развитое поддерживающие сообщество для решения возникающих проблем
3. Совместим с нестандартными/испорченными html и xml-файлами, в следствие чего является более универсальным
4. В своих структурах данных Beautiful Soup хранит только строки Unicode. Поэтому даже если файл лежит в другой кодировке, она будет изменена на Unicode самим BS, поэтому самому думать об этом не надо. (Но если в файле не определена кодировка, то он может сделать неверное предположение и использовать ошибочную кодировку Windows-1252)
5. BS по html файлу строит дерево синтаксического разбора, благодаря чему мы можем перемещаться к предкам и сыновьям. Объект парсера (экземпляр класса BeautifulSoup или BeautifulStoneSoup) обладает большой глубиной вложенности связанных структур данных
6. Поскольку методы поиска Beautiful Soup всегда определяют аргумент name, мы не можем использовать именованный аргумент с именем name. В качестве именованного аргумента также нельзя использовать зарезервированные слова Python, такие как for.


## 3. Изучаем структуру двух сайтов (Хабр, Циан) 

### Habr
Расмотрим, как выглядит ссылка на сайт, чтобы сгенерировать тесты: Каждая статья лежит по адресу "https://habr.com/ru/post/{0}/", где {0} -- ее номер. Таким образом, чтобы получить тесты мы можем проитерироваться по номера постов и получить то, что мы хотели.

Сайт имеет классическую структуру. Вся статья расположена в теге \<main>\. Имеет глубокую вложенность элементов. У каждого элемента заранее определен класс, к которому он принадлежит, что упрощает парсинг страницы. 
С данного сайта для каждой статьи мы хотим получить: теги, хабы, количество сохранений и время публикации. Заметим, что на одной странице одна статься, поэтому нам будет достаточно найти все использования классов "tm-article-snippet__datetime-published" – время публикации, "tm-tags-list__link" – теги, "tm-hubs-list__link" – хабы и "bookmarks-button__counter" – количество сохранений

### Cian 
Ссылка на сайт: 'https://spb.cian.ru/kupit-{0}-komnatnuyu-kvartiru/' – Сайт имеет поддомен для каждого города, например spb.cian, кроме Москвы, для нее используется просто "cian". Далее, если запрос стандартный, то под него выделен специальный url, как в примере "kupit-{0}-komnatnuyu-kvartiru", где {0} – количество комнат, если выбрано много фильтров, то будут прописаны параметры через &. 

Ключевая особенность сайта, что название классов в html-документе генерируется автоматически, поэтому мы не можем парсить по названию класса. У каждого элемента есть атрибут, выбранный разработчиком: data-name, по нему мы и будет находить нужные элементы. Мы хотим получить информацию о названии, геолокации и стоимости для каждого объявления. 
Каждое объявление лежит в \<div data-name="CardComponent">. После разбора на объявления из каждого получаем: \<<data-name="TitleComponent">> – название объявления, \<<data-mark="MainPrice">> – его цену, а также \<<data-name="GeoLabel">> – адрес. 

## 4. Каких возможностей библиотек не хватает

### Scrapy
1. На динамически загружающиеся данные есть ограничения, которые необходимо обходить, используя другие библиотеки
2. Может работать некорректно, если названия классов генерируются автоматически при каждой загрузке с помощью javascript
3. Неоднозначная обработка URL при заранее определенных правилах (может неоднократно ходить по URL, которые есть в стоп-листе, при этом не начиная работу с разрешенными URL)

### BeautifulSoup 
1. Библиотека изначально не знает о существовании самозакрывающихся тегов. О них надо специально сообщать и прописывать.
2. Достаточно медленная библиотека, чтобы ускорить время ее работы предлагается парсить только часть файла, которая нам интересна, или использовать многопоточность
3. Потребляет много памяти при парсинге.
4. Не умеет отправлять запросы на сайты, поэтому приходится пользоваться другими библиотеками, такими как requests или urlib2.
5. Требуется внешний анализатор для анализа загруженных данных. Наиболее известные парсеры - это XML-parser lxml, HTML-parser lxml, HTML5lib, html.parser.